[View in Colaboratory](https://colab.research.google.com/github/youqad/Neurorobotics_Project/blob/master/Inferring_Space_from_Sensorimotor_Dependencies.ipynb)

# Inferring Space from Sensorimotor Dependencies


## The Algorithm

1. Proprioceptive input is separated from exteroceptive input by noting that proprioceptive input remains silent when no motor commands are given, whereas exteroceptive input changes because of environmental change.

2. Estimation of the number of parameters needed to describe the variation in the exteroceptive inputs when only the environment changes. The algorithm issues no motor commands and calculates the covariance matrix of the observed environment-induced variations in sensory inputs. The dimension estimation is done by considering the eigenvalues of this covariance matrix. The eigenvalues $λ_i$ should fall into two classes: a class with values all equal to zero and a class with nonzero values. The two classes are separated by a clustering method (e.g. PCA). The number of nonzero eigenvalues is taken as the number of dimensions.

3. Estimation of the number of parameters needed to describe the variation in the exteroceptive inputs when only the body moved. The environment is kept fixed, and the algorithm gives random motor commands. The covariance matrix of the resulting changes is observed and the dimension is estimated from the number of nonzero eigenvalues in the same way as before.

4. Estimation of the number of parameters needed to describe the changes in exteroceptive inputs when both the body and the environment change. The environment is changed at random, and the organism gives random motor commands. The number of nonzero eigenvalues of the covariance matrix is obtained as before.

## Simulation

### Notations

Notation|Meaning
-|-
$$Q ≝ (Q_1, \ldots, Q_{3q})$$|positions of the joints
$$P ≝ (P_1, \ldots, P_{3p})$$|positions of the eyes
$$a^θ_i, a^φ_i, a^ψ_i$$|Euler angles for the orientation of eye i
$$Rot(a^θ_i, a^φ_i, a^ψ_i)$$|rotation matrix for eye i
$$C_{i,k}$$|relative position of photosensor $k$ within eye $i$
$$d ≝ (d_1, \ldots,d_p)$$|apertures of diaphragms
$$L ≝ (L_1,\ldots,L_{3r})$$|positions of the lights
$$θ ≝ (θ_1, \ldots, θ_r)$$|luminances of the lights
$$S^e_{i,k}$$|sensory input from exteroceptive sensor $k$ of eye $i$
$$S^p_i$$|sensory input from proprioceptive sensor $i$
$$M, E$$|motor command and environmental control vector


### Computing the sensory inputs


$$
\begin{align*}
(Q,P,a) &≝ σ(W_1 · σ(W_2 · M − μ_2)−μ_1)\\
L &≝ σ(V_1 ·σ(V_2 · E − ν_2) − ν_1)\\
∀1≤ k ≤ p', 1≤i≤p, \quad S^e_{i,k} &≝ d_i \sum\limits_{ j } \frac{θ_j}{\Vert P_i + Rot(a_i^θ, a_i^φ, a_i^ψ) \cdot C_{i,k} - L_j \Vert^2}\\
(S^p_i)_{1≤ i ≤ q'q} &≝ σ(U_1 · σ(U_2 · Q − τ_2) − τ_1)
\end{align*}
$$

where

- $W_1, W_2, V_1, V_2, U_1, U_2$ are matrices with coefficients drawn randomly from a uniform distribution between $−1$ and $1$
- the vectors $μ_1, μ_2, ν_1, ν_2, τ_1, τ_2$ too
- $σ$ is an arbitrary nonlinearity (e.g. the hyperbolic tangent function)
- the $C_{i,k}$ are drawn from a centered normal distribution whose variance (which can be understood as the size of the retina) is so that the sensory changes resulting from a rotation of the eye are of the same order of magnitude as the ones resulting from a translation of the eye
- $θ$ and $d$ are constants drawn at random in the interval $[0.5, 1]$


In [1]:
# /!\ We're using Python 3

import plotly
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
from scipy.spatial import distance

from sklearn import preprocessing

%matplotlib inline

plotly.offline.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML, Markdown
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

# Colorscales
def colorscale_list(cmap, number_colors, return_rgb_only=False):
    cm = plt.get_cmap(cmap)
    colors = [np.array(cm(i/number_colors)) for i in range(1, number_colors+1)]
    rgb_colors_plotly = []
    rgb_colors_only = []
    for i, c in enumerate(colors):
        col = 'rgb{}'.format(tuple(255*c[:-1]))
        rgb_colors_only.append(col)
        rgb_colors_plotly.append([i/number_colors, col])
        rgb_colors_plotly.append([(i+1)/number_colors, col])
    return rgb_colors_only if return_rgb_only else rgb_colors_plotly

from scipy.io import loadmat, whosmat
from numpy.random import randint

def formatted(f): 
    return format(f, '.2f').rstrip('0').rstrip('.')

## Organism 1

**Parameter**|**Value**
-|-
Dimension of motor commands|$40$
Dimension of environmental control vector|$40$
Dimension of proprioceptive inputs|$16 \quad (= 4×4)$
Dimension of exteroceptive inputs|$40 \quad (= 2 × 20)$
Number of eyes|$2$
Number of joints|$4$
Diaphragms|None
Number of lights|$3$
Light luminance|Fixed


1. The arm has

    - $4$ joints

        - each of which has $4$ **proprioceptive** sensors (whose output depended on the position of the joint)
    - $2$ eyes (for each of them: $3$ spatial and $3$ orientation coordinates)

        - on which there are $20$ omnidirectionally sensitive photosensors (**exteroceptive**)
2. the motor command is $40$-dimensional  

3. the environment consists of:

    - $3$ lights ($3$ spatial coordinates and $3$ luminance values for each of them)

In [4]:
!pip install -e git+https://github.com/youqad/Neurorobotics_Project.git#egg=sensorimotor_dependencies

Obtaining sensorimotor_dependencies from git+https://github.com/youqad/Neurorobotics_Project.git#egg=sensorimotor_dependencies
  Updating ./src/sensorimotor-dependencies clone
  Found existing installation: sensorimotor-dependencies 0.0.1
    Can't uninstall 'sensorimotor-dependencies'. No files were found to uninstall.
  Running setup.py develop for sensorimotor-dependencies


In [0]:
from sensorimotor_dependencies import organism1

In [0]:
#@title Parameters

M_size = 40 #@param {type:"slider", min:10, max:100, step:5}
E_size = 40 #@param {type:"slider", min:10, max:100, step:5}

# Number of Joints / q
nb_joints = 4 #@param {type:"slider", min:3, max:5, step:1}

# Number of eyes / p
nb_eyes = 2 #@param {type:"slider", min:2, max:5, step:1}

# Number of lights / r
nb_lights = 3 #@param {type:"slider", min:2, max:5, step:1}

# Number of exteroceptive photosensors / p'
extero = 20 #@param {type:"slider", min:15, max:25, step:1}

# Number of proprioceptive sensors / q'
proprio = 4 #@param {type:"slider", min:3, max:5, step:1}


# Sensory inputs were generated from...
nb_generating_motor_commands = 50 #@param {type:"slider", min:10, max:200, step:10}
nb_generating_env_positions = 50 #@param {type:"slider", min:10, max:200, step:10}

# Neighborhood size of the linear approximation:
# Motor commands/Environmental positions drawn from normal distribution
# with mean zero and standard deviation... 
neighborhood_size = 1e-8
# (Coordinates differing from 0 by more than the std deviation are set equal to 0)

sigma = np.tanh

In [25]:
seed = 3
sim = []

for var in np.arange(.1, 16.1, .1):
  val, bod, env, env_bod = Organism1(seed=seed, retina_size=var).get_dimensions()
  print("var = {:.1f} / dim of compensated mov. = {} ({} {} {})".format(var, val, bod, env, env_bod))
  sim.append(val)

sim = np.array(sim)
best_sim = np.argmax(sim)

var = 0.1 / dim of compensated mov. = 0 (5 1 6)
var = 0.2 / dim of compensated mov. = 0 (5 1 6)
var = 0.3 / dim of compensated mov. = 0 (5 1 6)
var = 0.4 / dim of compensated mov. = 0 (5 1 6)
var = 0.5 / dim of compensated mov. = 0 (5 1 6)
var = 0.6 / dim of compensated mov. = 0 (5 1 6)
var = 0.7 / dim of compensated mov. = 0 (5 1 6)
var = 0.8 / dim of compensated mov. = 0 (5 1 6)
var = 0.9 / dim of compensated mov. = 0 (5 1 6)
var = 1.0 / dim of compensated mov. = 0 (5 1 6)
var = 1.1 / dim of compensated mov. = 0 (5 1 6)
var = 1.2 / dim of compensated mov. = 0 (5 1 6)
var = 1.3 / dim of compensated mov. = 0 (5 1 6)
var = 1.4 / dim of compensated mov. = 0 (5 1 6)
var = 1.5 / dim of compensated mov. = 0 (5 1 6)
var = 1.6 / dim of compensated mov. = 0 (5 1 6)
var = 1.7 / dim of compensated mov. = 0 (5 1 6)
var = 1.8 / dim of compensated mov. = 0 (5 1 6)
var = 1.9 / dim of compensated mov. = 0 (5 1 6)
var = 2.0 / dim of compensated mov. = 0 (5 1 6)
var = 2.1 / dim of compensated mov. = 0 

var = 6.4 / dim of compensated mov. = 1 (6 1 6)
var = 6.5 / dim of compensated mov. = 1 (6 1 6)
var = 6.6 / dim of compensated mov. = 1 (6 1 6)
var = 6.7 / dim of compensated mov. = 1 (6 1 6)
var = 6.8 / dim of compensated mov. = 3 (6 3 6)
var = 6.9 / dim of compensated mov. = 3 (6 3 6)
var = 7.0 / dim of compensated mov. = 1 (6 1 6)
var = 7.1 / dim of compensated mov. = 3 (6 3 6)
var = 7.2 / dim of compensated mov. = 3 (6 3 6)
var = 7.3 / dim of compensated mov. = 3 (6 3 6)
var = 7.4 / dim of compensated mov. = 3 (6 3 6)
var = 7.5 / dim of compensated mov. = 3 (6 3 6)
var = 7.6 / dim of compensated mov. = 3 (6 3 6)
var = 7.7 / dim of compensated mov. = 1 (6 1 6)
var = 7.8 / dim of compensated mov. = 3 (6 3 6)
var = 7.9 / dim of compensated mov. = 3 (6 3 6)
var = 8.0 / dim of compensated mov. = 3 (6 3 6)
var = 8.1 / dim of compensated mov. = 3 (6 3 6)
var = 8.2 / dim of compensated mov. = 3 (6 3 6)
var = 8.3 / dim of compensated mov. = 3 (6 3 6)
var = 8.4 / dim of compensated mov. = 1 

var = 12.7 / dim of compensated mov. = 2 (7 1 6)
var = 12.8 / dim of compensated mov. = 2 (7 1 6)
var = 12.9 / dim of compensated mov. = 2 (7 1 6)
var = 13.0 / dim of compensated mov. = 2 (7 1 6)
var = 13.1 / dim of compensated mov. = 2 (7 1 6)
var = 13.2 / dim of compensated mov. = 2 (7 1 6)
var = 13.3 / dim of compensated mov. = 2 (7 1 6)
var = 13.4 / dim of compensated mov. = 2 (7 1 6)
var = 13.5 / dim of compensated mov. = 2 (7 1 6)
var = 13.6 / dim of compensated mov. = 2 (7 1 6)
var = 13.7 / dim of compensated mov. = 2 (7 1 6)
var = 13.8 / dim of compensated mov. = 2 (7 1 6)
var = 13.9 / dim of compensated mov. = 2 (7 1 6)
var = 14.0 / dim of compensated mov. = 2 (7 1 6)
var = 14.1 / dim of compensated mov. = 2 (7 1 6)
var = 14.2 / dim of compensated mov. = 2 (7 1 6)
var = 14.3 / dim of compensated mov. = 2 (7 1 6)
var = 14.4 / dim of compensated mov. = 2 (7 1 6)
var = 14.5 / dim of compensated mov. = 2 (7 1 6)
var = 14.6 / dim of compensated mov. = 2 (7 1 6)
var = 14.7 / dim of 

In [0]:
# Other Dimension Reduction Methods
## LDA is not applicable for our data
## Found codes for classical MDS so far -- 
## to be cordinated with our data

from __future__ import division
def mds(D):
    """                                                                                       
    Classical multidimensional scaling (MDS)                                                  
                                                                                               
    Parameters                                                                                
    ----------                                                                                
    D : (n, n) array                                                                          
        Symmetric distance matrix.                                                            
                                                                                               
    Returns                                                                                   
    -------                                                                                   
    Y : (n, p) array                                                                          
        Configuration matrix. Each column represents a dimension. Only the                    
        p dimensions corresponding to positive eigenvalues of B are returned.                 
        Note that each dimension is only determined up to an overall sign,                    
        corresponding to a reflection.                                                        
                                                                                               
    e : (n,) array                                                                            
        Eigenvalues of B.                                                                     
                                                                                               
    """
    # Number of points                                                                        
    n = len(D)
 
    # Centering matrix                                                                        
    H = np.eye(n) - np.ones((n, n))/n
 
    # YY^T                                                                                    
    B = -H.dot(D**2).dot(H)/2
 
    # Diagonalize                                                                             
    evals, evecs = np.linalg.eigh(B)
 
    # Sort by eigenvalue in descending order                                                  
    idx   = np.argsort(evals)[::-1]
    evals = evals[idx]
    evecs = evecs[:,idx]
 
    # Compute the coordinates using positive-eigenvalued components only                      
    w, = np.where(evals > 0)
    L  = np.diag(np.sqrt(evals[w]))
    V  = evecs[:,w]
    Y  = V.dot(L)
 
    return Y, evals


#Autoencoder: https://github.com/asdspal/dimRed/blob/master/autoencoder.ipynb